In [1]:
"OM NAMO BHAGVATE VASUDEVAY"

'OM NAMO BHAGVATE VASUDEVAY'

In [3]:
!pip install openpyxl

In [11]:
import requests
from bs4 import BeautifulSoup
import nltk
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
import re
# Download required NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Function to load positive and negative words
def load_words(file_path):
    with open(file_path, "r", encoding="ISO-8859-1", errors="ignore") as file:
        return set(word.strip().lower() for word in file.readlines())

# Function to scrape text from a URL
def scrape_text(url):
    response = requests.get(url)
    response.raise_for_status()  # Check for a successful response
    soup = BeautifulSoup(response.text, "html.parser")

    # Extract title and main article text
    title = soup.title.get_text() if soup.title else ""
    paragraphs = soup.find_all("p")
    article_text = " ".join(p.get_text() for p in paragraphs)

    return title + " " + article_text

# Function to preprocess and tokenize text
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    tokens = word_tokenize(text)  # Tokenize the text
    tokens = [word for word in tokens if word.isalpha()]  # Remove non-alphabetic tokens
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]  # Remove stop words
    return tokens
def count_sentences(text):
    # Tokenize the text into sentences
    sentences = sent_tokenize(text)
    # Return the number of sentences
    return len(sentences)
def average_word_length(tokens):      
    # Calculate the total number of characters in all words
    total_characters = sum(len(word) for word in tokens)    
    # Calculate the total number of words
    total_words = len(tokens)    
    # Avoid division by zero if there are no words
    if total_words == 0:
        return 0    
    # Calculate and return the average word length
    return total_characters / total_words    
def average_sentence_length(text):
    # Tokenize the text into sentences
    sentences = sent_tokenize(text)    
    total_length = sum(len(sentence.split()) for sentence in sentences)
    # Avoid division by zero if there are no sentences
    if len(sentences) == 0:
        return 0
    # Calculate and return the average sentence length
    return total_length / len(sentences)
def count_syllables(token):
    # Regular expression to match vowels
    vowels = "aeiouy"
    # If word ends in "es" or "ed" and has more than one syllable, ignore the last syllable
    if token.endswith("es") or token.endswith("ed"):
        token = re.sub(r"(es|ed)$", "", token)
    # Count the vowels in the word as syllables
    syllable_count = sum(1 for char in token if char in vowels)
    # Ensure there's at least one syllable per word
    return max(1, syllable_count)    
def count_personal_pronouns(text):
    # Convert text to lowercase for uniformity
    text = text.lower()
    
    # Define the list of personal pronouns to search for
    personal_pronouns = ["i", "we", "my", "ours", "us"]
    
    # Use regular expressions to find exact matches, ignoring "us" if it's uppercase (i.e., "US")
    counts = {pronoun: len(re.findall(rf"\b{pronoun}\b", text)) for pronoun in personal_pronouns}
    
    # Exclude "US" if it appears in uppercase
    counts["us"] -= len(re.findall(r"\bUS\b", text))
    
    return counts

# Main function to process each URL and generate output
def process_urls(input_file, positive_file, negative_file,output_file):
    # Load positive and negative word lists
    positive_words = load_words(positive_file)
    negative_words = load_words(negative_file)

    # Load input data from Excel
    df = pd.read_excel(input_file)
    results = []

    # Process each row in the input file
    for index, row in df.iterrows():
        url_id = row['URL_ID']
        url_link = row['URL']
        print("url_id",url_id)
        print("url_link",url_link)

        try:
            # Scrape and process text from URL
            text = scrape_text(url_link)
            tokens = preprocess_text(text)
            Total_tokens=len(tokens)
            #Sentence count
            sentence_count = count_sentences(text)
            #Average word length
            avg_word_length = round(average_word_length(text), 2)

            # Count positive and negative tokens
            positive_score = sum(1 for token in tokens if token in positive_words)
            negative_score = sum(1 for token in tokens if token in negative_words)
            #Calculate polarity score
            polarity_score = (positive_score -negative_score )/ ((positive_score +negative_score ) + 0.000001)
            #Subjectivity Score
            subjectivity_score = (positive_score + negative_score)/ ((Total_tokens) + 0.000001)
            #Average Number of Words Per Sentence
            avg_word_per_sent = len(tokens) / sentence_count
            #Average sentence length 
            avg_sent_length=average_sentence_length(text)
            #Syllable counts
            syllable_counts = {token: count_syllables(token) for token in tokens}
            #complex word count
            complex_word_count = sum(1 for token in tokens if count_syllables(token) > 2)
            #Percentage of Complex words 
            percent_of_complex_words= complex_word_count/len(tokens)
            #FOG INDEX
            Fog_index = 0.4 * (avg_sent_length +percent_of_complex_words )
            #Word count
            word_count=len(tokens)
            #PERSONAL PRONOUNS
            personal_pronouns=count_personal_pronouns(text)
            # Append result for each URL
            results.append({
                "url_id": url_id,
                "url_link": url_link,
                "positive_score":positive_score ,
                "negative_score":negative_score,
                "polarity_score":polarity_score,
                "subjectivity_score":subjectivity_score,
                "sentence_count":sentence_count,
                "avg_word_length":avg_word_length,
                "word_count":word_count,
                "personal_pronouns":personal_pronouns,
                "Fog_index":Fog_index,
                "percent_of_complex_words":percent_of_complex_words,
                "complex_word_count":complex_word_count,
                "syllable_counts":syllable_counts,
                "avg_sent_length":avg_sent_length,
                "avg_word_per_sent":avg_word_per_sent,
                
            })
        except Exception as e:
            print(f"Failed to process URL {url_link}: {e}")
            results.append({
                "url_id": url_id,
                "url_link": url_link,
                "positive_scor": None,
                "negative_score": None,
                "polarity_score":None,
                "subjectivity_score":None,
                "sentence_count":None,
                "avg_word_length":None,
                "word_count":None,
                "personal_pronouns":None,
                "Fog_index":None,
                "percent_of_complex_words":None,
                "complex_word_count":None,
                "syllable_counts":None,
                "avg_sent_length":None,
                "avg_word_per_sent":None,
            })

    # Save results to an output Excel file
    output_df = pd.DataFrame(results)
    output_df.to_excel(output_file, index=False)
    
# File paths
input_file = r"C:\Users\LENOVO\Desktop\Input.xlsx"
positive_file = r"C:\Users\LENOVO\Desktop\positive-words.txt"
negative_file = r"C:\Users\LENOVO\Desktop\negative-words.txt"
output_file = "output.xlsx"

# Run the processing function
process_urls(input_file, positive_file, negative_file,output_file)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


url_id Netclan20241017
url_link https://insights.blackcoffer.com/ai-and-ml-based-youtube-analytics-and-content-creation-tool-for-optimizing-subscriber-engagement-and-content-strategy/
url_id Netclan20241018
url_link https://insights.blackcoffer.com/enhancing-front-end-features-and-functionality-for-improved-user-experience-and-dashboard-accuracy-in-partner-hospital-application/
url_id Netclan20241019
url_link https://insights.blackcoffer.com/roas-dashboard-for-campaign-wise-google-ads-budget-tracking-using-google-ads-ap/
url_id Netclan20241020
url_link https://insights.blackcoffer.com/efficient-processing-and-analysis-of-financial-data-from-pdf-files-addressing-formatting-inconsistencies-and-ensuring-data-integrity-for-a-toyota-dealership-management-firm/
url_id Netclan20241021
url_link https://insights.blackcoffer.com/development-of-ea-robot-for-automated-trading/
url_id Netclan20241022
url_link https://insights.blackcoffer.com/ai-and-ml-based-youtube-analytics-and-content-creation-to